In [7]:
import numpy as np
# import qit
import matplotlib
import matplotlib.pyplot as plt
from random import randint
import pycosat
import random
import scipy.linalg as la
import pandas as pd
import scipy.sparse as sparse
import scipy.sparse.linalg as lasp
from numba import jit
import pymp
import os


In [2]:

def random_pick(some_list, probabilities):
    x = random.uniform(0, 1)
    cumulative_probability = 0.0
    for item, item_probability in zip(some_list, probabilities):
        cumulative_probability += item_probability
        if x < cumulative_probability: break
    return item



In [3]:
# function to generate randomly 3-SAT with n_variables and m_clauses
def cnf_random_generator(n_variables,m_clauses):
    cnf=[]
    for i in range(m_clauses):
        a=np.array(sorted(random.sample(range(1,n_variables+1),3)))
        b=np.array([(-1)**randint(0,1),(-1)**randint(0,1),(-1)**randint(0,1)])
        cnf.append(a*b)
    return cnf

# Define projectors as diagonal matrix - array
P_0=np.array([1,0])
P_1=np.array([0,1])
I=np.array([1,1])

def find_solution(cnf):
    cnf_list = list(map(lambda x: x.tolist(), cnf))
    sol = pycosat.solve(cnf_list)
    if sol == 'UNSAT':
        sol = 0
    elif sol == 'UNKNOWN':
        sol = 0
    else:
        sol = 1
    return sol

    
def obtain_samples(N, eH, samples, string):
    data = []
    for i in range(samples):
        randpick = random_pick(list(range(2**N)), eH)
        format_randpick = list(map(int, list(format(randpick, string))))
        data.extend(format_randpick)
    return data
    


# function transforms clause to Hamiltonian
def clause_to_hamiltonian(clause,n_variables):
    if np.sign(clause[0])==1:
        H=P_0
    else:
        H=P_1
    for i in range(1,np.abs(clause[0])):
        H=sparse.kron(I,H)
    
    if np.sign(clause[1])==1:
        H_1=P_0
    else:
        H_1=P_1        
    for i in range(np.abs(clause[0])+1,np.abs(clause[1])):
        H=sparse.kron(H,I)
    H=sparse.kron(H,H_1)
    
    if np.sign(clause[2])==1:
        H_2=P_0
    else:
        H_2=P_1        
    for i in range(np.abs(clause[1])+1,np.abs(clause[2])):
        H=sparse.kron(H,I)
    H=sparse.kron(H,H_2)
    
    for i in range(np.abs(clause[2])+1, n_variables+1):
        H=sparse.kron(H,I)
    return H

# function transforms 3-SAT from CNF form to Hamiltonian, just the sum of all clauses operators
def cnf_to_hamiltonian(cnf, n_variables):
    H=clause_to_hamiltonian(cnf[0],n_variables)
    for i in range(1,len(cnf)):
        H+=clause_to_hamiltonian(cnf[i],n_variables)
    return H

def cnf_to_hamiltonian_array(cnf, n_variables):
    H=clause_to_hamiltonian(cnf[0],n_variables)
    for i in range(1,len(cnf)):
        H+=clause_to_hamiltonian(cnf[i],n_variables)
    
    return H.toarray()[0,:]


def hamiltonian_to_thermal(H, beta):
    U=np.exp(-beta*H)
    U = U/np.sum(U)
    return U


# calculating sum of ground states of 3-SAT Hamiltonian
def ground_state(H, n_variables):
#   from cnf to Hamiltonian represented array, since it diagonal
#     find minimal eigenvalue and the superposition of ground states
    min_energy=np.where(H == H.min())[0]
#   mixinf of all grround states
    ground_state=np.zeros(2**n_variables)
    for i in min_energy:
        ground_state[i]=1
    return ground_state



# evolution function
# initial_state=initial_state(10)




def evolution(cnf,n_variables, beta):
#     generate superposition of ground states

    H=cnf_to_hamiltonian(cnf,n_variables).toarray()[0,:]
    g_state=ground_state(H,n_variables)
#     generate initial state
#     i_state=initial_state
#     generate cooling operator
    U=np.exp(-beta*H)
    
    return np.matmul(g_state, U.T/np.sum(U))  

In [ ]:
%%time
## alpha = clauses/variables = M/N


N = 15
#beta = 5
Hams_per_M = 10
samples_per_ham = 30


clauses_array = []
variable_array = []
sol_array = []
stringvar = '0' + str(N) + 'b'
cnf_array = []
beta_array = []


#betas = np.linspace(0, 3, 5)
betas = np.linspace(0, 4.5,10)


n_betas = len(betas)


folder = './data_' + '_N' + str(N) + '_HpM' + str(Hams_per_M) + '_SpH' + str(samples_per_ham) + '/'   
if not os.path.exists(folder):
    os.makedirs(folder)


print('start')
with pymp.Parallel(6) as p:
    for j, beta in enumerate(betas): 
        for M in range(1, 20*N):
            for i in range(Hams_per_M):
                print('beta = ' + str(j + 1) +  ' / '  + str(n_betas) + ' ---- M = ' + str(M) + ' / ' 
                      + str(20*N -1) + ' ---- H per M = ' + str(i) + ' / ' + str(Hams_per_M-1))
                cnf = cnf_random_generator(N,M)
                sol = find_solution(cnf)
                H = cnf_to_hamiltonian_array(cnf, N)
                eH = hamiltonian_to_thermal(H, beta)  
                data = obtain_samples(N, eH, samples_per_ham, stringvar)
                variable_array.append(data)
                sol_array.append(sol)
                cnf_array.append(cnf)
                clauses_array.append(M)

        
        string_variable_array = folder + 'variable_array_b' + str(beta) + '_N' + str(N) + '_HpM' + str(Hams_per_M) + '_SpH' + str(samples_per_ham) +  '.npy'         
        string_sol_array = folder + 'sol_array_b' + str(beta)  + '_N' + str(N)  + '_HpM' + str(Hams_per_M) + '_SpH' + str(samples_per_ham) + '.npy'
        string_cnf_array = folder + 'cnf_array_b' + str(beta)  + '_N' + str(N) + '_HpM' + str(Hams_per_M) + '_SpH' + str(samples_per_ham) + '.npy'
        string_clauses_array = folder + 'clauses_array_b' + str(beta)  + '_N' + str(N) + '_HpM' + str(Hams_per_M) + '_SpH' + str(samples_per_ham) + '.npy'
    
    
        np.save(string_variable_array, np.array(variable_array))
        np.save(string_sol_array, np.array(sol_array))
        np.save(string_cnf_array, np.array(cnf_array))
        np.save(string_clauses_array, np.array(clauses_array))

        
#print(cnf_array)        
#print(variable_array)
        
        

start
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 0 / 9
start
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 0 / 9
start
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 0 / 9
start
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 0 / 9
start
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 1 / 9
start
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 1 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 1 / 2

beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 4 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 3 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 4 / 299 ---- H per M = 1 / 9
beta = 1 / 1

beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 6 / 299 ---- H per M = 4 / 9
beta = 1 / 1

beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 8 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 9 / 299 ---- H per M = 3 / 9
beta = 1 / 1

beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 12 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 12 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 12 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 12 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 11 / 299 ---- H per M =

...

beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 167 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 171 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 171 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 171 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 167 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 171 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 171 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 171 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 171 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 167 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 171 /

beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 170 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 173 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 173 /

beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 172 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 172 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 176 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 172 /

beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 5 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 175 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 6 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 175 /

beta = 1 / 10 ---- M = 180 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 180 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 177 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 180 / 299 ---- H per M = 7 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 177 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 2 / 9
beta = 1 / 10 ---- M = 180 / 299 ---- H per M = 8 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 1 / 9
beta = 1 / 10 ---- M = 178 / 299 ---- H per M = 0 / 9
beta = 1 / 10 ---- M = 180 / 299 ---- H per M = 9 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 3 / 9
beta = 1 / 10 ---- M = 181 / 299 ---- H per M = 4 / 9
beta = 1 / 10 ---- M = 181 /

In [97]:
n=5
m=4
beta = 5
samples = 2
cnf = cnf_random_generator(n,m)
print(cnf)
print()
H = cnf_to_hamiltonian_array(cnf, n)
print(H)
eH = hamiltonian_to_thermal(H, beta)
#print(eH)
#print(eH)
#list(range(2**n))
#randpick = random_pick(list(range(2**n)), eH)
#print(randpick)
stringvar = '0' + str(n) + 'b'
#print(format(randpick, stringvar))
obtain_samples(N, eH, samples, stringvar)

[array([ 1, -4,  5]), array([-2, -3,  5]), array([ 1, -3, -4]), array([-1,  2,  4])]

[0 0 1 0 0 0 2 1 0 0 1 0 1 0 3 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0]


[0, 1, 1, 1, 1, 0, 1, 1, 1, 1]

In [24]:
list(bin(5)[2:])

['1', '0', '1']

In [76]:
x= 10
print(format(x, '010b'))
list(map(int, list(format(x, '010b'))))

0000001010


[0, 0, 0, 0, 0, 0, 1, 0, 1, 0]

In [74]:
cnf = cnf_random_generator(n,m)
print(cnf)
#pycosat.solve(cnf)

#nf_list = map(np_to_list, cnf)
cnf_list = list(map(lambda x: x.tolist(), cnf))
print(cnf_list)
pycosat.solve(cnf_list)


[array([-1,  3,  5]), array([-1, -4,  5]), array([-1, -3,  5]), array([-3, -4, -5])]
[[-1, 3, 5], [-1, -4, 5], [-1, -3, 5], [-3, -4, -5]]


[-1, -2, -3, -4, 5]

In [83]:
pycosat.solve([[1,-2],[-1,2],[1,2], [-1,-2]])

'UNSAT'